In [ ]:
from pathlib import Path
import torch
import cv2
import json
import numpy as np
from LightGlue.lightglue import LightGlue, SuperPoint
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def load_and_prepare_image(image_path):
    image = load_image(image_path)
    return image

def calculate_matches(extractor, matcher, img0, img1):
    feats0 = extractor.extract(img0.to(device))
    feats1 = extractor.extract(img1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    return feats0, feats1, matches01

def adjust_keypoints(kpts, half_size, overlap):
    kpts_adjusted = kpts.detach().clone()
    kpts_adjusted[:, 1] += half_size - overlap
    return kpts_adjusted


In [ ]:
try:
    with open(json_path, 'r') as f:
        bounding_boxes_data = json.load(f)
except FileNotFoundError:
    print(f"Archivo JSON no encontrado: {json_path}")
    continue


In [ ]:
from concurrent.futures import ThreadPoolExecutor

def process_product_image(product_image_path):
    # Colocar aquí la lógica de procesamiento para cada imagen.
    pass

with ThreadPoolExecutor() as executor:
    for product_image_path in product_folder.glob('*'):
        if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']:
            executor.submit(process_product_image, product_image_path)


In [ ]:
with torch.no_grad():
    feats0_top = extractor.extract(product_image.to(device))
    feats1_top = extractor.extract(top_half.to(device))


In [ ]:
def load_and_prepare_image(image_path):
    image = load_image(image_path)
    return image

def calculate_matches(extractor, matcher, img0, img1):
    feats0 = extractor.extract(img0.to(device))
    feats1 = extractor.extract(img1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    return feats0, feats1, matches01

def adjust_keypoints(kpts, half_size, overlap):
    kpts_adjusted = kpts.detach().clone()
    kpts_adjusted[:, 1] += half_size - overlap
    return kpts_adjusted

def draw_bounding_boxes(image, bboxes):
    for bbox in bboxes:
        if (bbox[2] <= image.shape[1] and bbox[0] >= 0 and
            bbox[3] <= image.shape[0] and bbox[1] >= 0):
            cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)  # Rojo
    return image

def process_product_image(product_image_path, top_half, bottom_half, half_size, overlap, device):
    print(f"Analizando el producto: {product_image_path.name}")
    product_image = load_image(product_image_path)

    top_half_matches, bottom_half_matches = 0, 0

    feats0_top, feats1_top, matches01_top = calculate_matches(extractor, matcher, product_image, top_half)
    kpts0_top, kpts1_top, matches_top = feats0_top["keypoints"], feats1_top["keypoints"], matches01_top["matches"]
    top_half_matches = len(matches_top)

    feats0_bottom, feats1_bottom, matches01_bottom = calculate_matches(extractor, matcher, product_image, bottom_half)
    kpts0_bottom, kpts1_bottom, matches_bottom = feats0_bottom["keypoints"], feats1_bottom["keypoints"], matches01_bottom["matches"]
    bottom_half_matches = len(matches_bottom)

    adjusted_kpts1_bottom = adjust_keypoints(kpts1_bottom, half_size, overlap)

    print(f"Coincidencias en la mitad superior: {top_half_matches}, coincidencias en la mitad inferior: {bottom_half_matches}")
    return top_half_matches, bottom_half_matches

# Definir parámetros y configurar el dispositivo y modelos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)
matcher = LightGlue(features="superpoint").eval().to(device)

# Definir imágenes y procesar cada producto en paralelo
gondola_image_path = "/Users/manu/Downloads/Mayo-Junio-Julio (teoricamente) 2/39-MJJ-.jpeg"
gondola_split = load_and_prepare_image(gondola_image_path)
height, width = gondola_split.shape[1:3]
top_half_end = int(height * 0.6)
bottom_half_start = int(height * 0.4)
top_half = gondola_split[:, :top_half_end, :]
bottom_half = gondola_split[:, bottom_half_start:, :]
half_size = top_half_end

product_folder = Path("/Users/manu/Downloads/Cuidado Capilar/Tresseme")

with ThreadPoolExecutor() as executor:
    for product_image_path in product_folder.glob('*'):
        if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']:
            executor.submit(process_product_image, product_image_path, top_half, bottom_half, half_size, overlap, device)